In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import time
import datetime
import os
from models import  LinfPGDAttack, ResNet18

In [2]:
learning_rate = 0.1
epsilon = 0.0314
k = 7
alpha = 0.00784
file_name = 'basic_training'

device = 'cuda' if torch.cuda.is_available() else 'cpu'

transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

adversary = LinfPGDAttack(net,epsilon,alpha,k)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 10 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)

def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    benign_loss = 0
    adv_loss = 0
    benign_correct = 0
    adv_correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            total += targets.size(0)

            
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            benign_loss += loss.item()

            _, predicted = outputs.max(1)
            benign_correct += predicted.eq(targets).sum().item()

            if batch_idx % 10 == 0:
                print('\nCurrent batch:', str(batch_idx))
                print('Current benign test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
                print('Current benign test loss:', loss.item())

            adv = adversary.perturb(inputs, targets)
            adv_outputs = net(adv)
            loss = criterion(adv_outputs, targets)
            adv_loss += loss.item()

            _, predicted = adv_outputs.max(1)
            adv_correct += predicted.eq(targets).sum().item()

            if batch_idx % 10 == 0:
                print('Current adversarial test accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
                print('Current adversarial test loss:', loss.item())

    print('\nTotal benign test accuarcy:', 100. * benign_correct / total)
    print('Total adversarial test Accuarcy:', 100. * adv_correct / total)
    print('Total benign test loss:', benign_loss)
    print('Total adversarial test loss:', adv_loss)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')

def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 100:
        lr /= 10
    if epoch >= 150:
        lr /= 10
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [4]:
t1 = time.time()
for epoch in range(0, 200):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)
t2 = time.time()


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.09375
Current benign train loss: 2.4559214115142822

Current batch: 10
Current benign train accuracy: 0.09375
Current benign train loss: 4.990118503570557

Current batch: 20
Current benign train accuracy: 0.140625
Current benign train loss: 2.5589332580566406

Current batch: 30
Current benign train accuracy: 0.1171875
Current benign train loss: 2.260420560836792

Current batch: 40
Current benign train accuracy: 0.1015625
Current benign train loss: 2.229231357574463

Current batch: 50
Current benign train accuracy: 0.1875
Current benign train loss: 2.198711633682251

Current batch: 60
Current benign train accuracy: 0.1484375
Current benign train loss: 2.2247283458709717

Current batch: 70
Current benign train accuracy: 0.21875
Current benign train loss: 2.027625560760498

Current batch: 80
Current benign train accuracy: 0.2578125
Current benign train loss: 2.0409114360809326

Current batch: 90
Current benign train ac


Current batch: 170
Current benign train accuracy: 0.375
Current benign train loss: 1.5706628561019897

Current batch: 180
Current benign train accuracy: 0.4453125
Current benign train loss: 1.5571221113204956

Current batch: 190
Current benign train accuracy: 0.328125
Current benign train loss: 1.6942390203475952

Current batch: 200
Current benign train accuracy: 0.390625
Current benign train loss: 1.614717721939087

Current batch: 210
Current benign train accuracy: 0.421875
Current benign train loss: 1.542545199394226

Current batch: 220
Current benign train accuracy: 0.3984375
Current benign train loss: 1.579353928565979

Current batch: 230
Current benign train accuracy: 0.5
Current benign train loss: 1.5484009981155396

Current batch: 240
Current benign train accuracy: 0.359375
Current benign train loss: 1.5857712030410767

Current batch: 250
Current benign train accuracy: 0.4296875
Current benign train loss: 1.5132555961608887

Current batch: 260
Current benign train accuracy: 0.3


Current batch: 350
Current benign train accuracy: 0.546875
Current benign train loss: 1.1503087282180786

Current batch: 360
Current benign train accuracy: 0.609375
Current benign train loss: 1.1327219009399414

Current batch: 370
Current benign train accuracy: 0.6484375
Current benign train loss: 1.0408809185028076

Current batch: 380
Current benign train accuracy: 0.5078125
Current benign train loss: 1.1974977254867554

Current batch: 390
Current benign train accuracy: 0.4625
Current benign train loss: 1.3494714498519897

Total benign train accuarcy: 49.22
Total benign train loss: 542.6396272182465

[ Test epoch: 2 ]

Current batch: 0
Current benign test accuracy: 0.62
Current benign test loss: 1.1191695928573608
Current adversarial test accuracy: 0.05
Current adversarial test loss: 4.100703716278076

Current batch: 10
Current benign test accuracy: 0.57
Current benign test loss: 1.2740459442138672
Current adversarial test accuracy: 0.05
Current adversarial test loss: 4.2266254425048

Current adversarial test accuracy: 0.05
Current adversarial test loss: 5.002094745635986

Current batch: 70
Current benign test accuracy: 0.54
Current benign test loss: 1.281144380569458
Current adversarial test accuracy: 0.06
Current adversarial test loss: 5.233758926391602

Current batch: 80
Current benign test accuracy: 0.54
Current benign test loss: 1.2304176092147827
Current adversarial test accuracy: 0.01
Current adversarial test loss: 4.891105651855469

Current batch: 90
Current benign test accuracy: 0.61
Current benign test loss: 1.0779640674591064
Current adversarial test accuracy: 0.02
Current adversarial test loss: 4.760003566741943

Total benign test accuarcy: 54.49
Total adversarial test Accuarcy: 3.04
Total benign test loss: 126.3501044511795
Total adversarial test loss: 508.37612438201904
Model Saved!

[ Train epoch: 4 ]

Current batch: 0
Current benign train accuracy: 0.6953125
Current benign train loss: 0.8652152419090271

Current batch: 10
Current benign train accurac


Current batch: 100
Current benign train accuracy: 0.609375
Current benign train loss: 1.0073024034500122

Current batch: 110
Current benign train accuracy: 0.59375
Current benign train loss: 1.2257113456726074

Current batch: 120
Current benign train accuracy: 0.65625
Current benign train loss: 0.9237487316131592

Current batch: 130
Current benign train accuracy: 0.625
Current benign train loss: 1.0055031776428223

Current batch: 140
Current benign train accuracy: 0.703125
Current benign train loss: 0.9023586511611938

Current batch: 150
Current benign train accuracy: 0.5859375
Current benign train loss: 1.0145272016525269

Current batch: 160
Current benign train accuracy: 0.6953125
Current benign train loss: 0.9151454567909241

Current batch: 170
Current benign train accuracy: 0.671875
Current benign train loss: 0.8075334429740906

Current batch: 180
Current benign train accuracy: 0.7578125
Current benign train loss: 0.715360701084137

Current batch: 190
Current benign train accuracy

...

Current adversarial test accuracy: 0.0
Current adversarial test loss: 22.515806198120117

Current batch: 70
Current benign test accuracy: 0.92
Current benign test loss: 0.43816304206848145
Current adversarial test accuracy: 0.01
Current adversarial test loss: 21.791423797607422

Current batch: 80
Current benign test accuracy: 0.95
Current benign test loss: 0.2134997397661209
Current adversarial test accuracy: 0.0
Current adversarial test loss: 22.733125686645508

Current batch: 90
Current benign test accuracy: 0.94
Current benign test loss: 0.19883695244789124
Current adversarial test accuracy: 0.0
Current adversarial test loss: 22.86884117126465

Total benign test accuarcy: 94.86
Total adversarial test Accuarcy: 0.1
Total benign test loss: 21.176285271067172
Total adversarial test loss: 2219.7291297912598
Model Saved!

[ Train epoch: 194 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0010399931343272328

Current batch: 10
Current benign train accur


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0010115592740476131

Current batch: 110
Current benign train accuracy: 1.0
Current benign train loss: 0.0008131839567795396

Current batch: 120
Current benign train accuracy: 1.0
Current benign train loss: 0.0010204475838690996

Current batch: 130
Current benign train accuracy: 1.0
Current benign train loss: 0.0016844787169247866

Current batch: 140
Current benign train accuracy: 1.0
Current benign train loss: 0.001350192236714065

Current batch: 150
Current benign train accuracy: 1.0
Current benign train loss: 0.0011199491564184427

Current batch: 160
Current benign train accuracy: 1.0
Current benign train loss: 0.0007430891855619848

Current batch: 170
Current benign train accuracy: 1.0
Current benign train loss: 0.001376418978907168

Current batch: 180
Current benign train accuracy: 1.0
Current benign train loss: 0.0008632302633486688

Current batch: 190
Current benign train accuracy: 1.0
Current ben


Current batch: 280
Current benign train accuracy: 1.0
Current benign train loss: 0.001257437514141202

Current batch: 290
Current benign train accuracy: 1.0
Current benign train loss: 0.0005371444276534021

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0006275334162637591

Current batch: 310
Current benign train accuracy: 1.0
Current benign train loss: 0.00048424964188598096

Current batch: 320
Current benign train accuracy: 1.0
Current benign train loss: 0.0010827918304130435

Current batch: 330
Current benign train accuracy: 1.0
Current benign train loss: 0.001561289420351386

Current batch: 340
Current benign train accuracy: 1.0
Current benign train loss: 0.0005491047631949186

Current batch: 350
Current benign train accuracy: 1.0
Current benign train loss: 0.0008948601898737252

Current batch: 360
Current benign train accuracy: 1.0
Current benign train loss: 0.001279747230000794

Current batch: 370
Current benign train accuracy: 1.0
Current ben


Current batch: 30
Current benign test accuracy: 0.94
Current benign test loss: 0.19261880218982697
Current adversarial test accuracy: 0.01
Current adversarial test loss: 21.501676559448242

Current batch: 40
Current benign test accuracy: 0.92
Current benign test loss: 0.2191634178161621
Current adversarial test accuracy: 0.0
Current adversarial test loss: 22.373851776123047

Current batch: 50
Current benign test accuracy: 0.97
Current benign test loss: 0.09809225797653198
Current adversarial test accuracy: 0.0
Current adversarial test loss: 22.440053939819336

Current batch: 60
Current benign test accuracy: 0.96
Current benign test loss: 0.224912628531456
Current adversarial test accuracy: 0.0
Current adversarial test loss: 21.999771118164062

Current batch: 70
Current benign test accuracy: 0.92
Current benign test loss: 0.4710344672203064
Current adversarial test accuracy: 0.0
Current adversarial test loss: 21.87213897705078

Current batch: 80
Current benign test accuracy: 0.95
Curre


Current batch: 40
Current benign train accuracy: 1.0
Current benign train loss: 0.003833515802398324

Current batch: 50
Current benign train accuracy: 1.0
Current benign train loss: 0.000933258153963834

Current batch: 60
Current benign train accuracy: 1.0
Current benign train loss: 0.0010951958829537034

Current batch: 70
Current benign train accuracy: 1.0
Current benign train loss: 0.0010959364008158445

Current batch: 80
Current benign train accuracy: 1.0
Current benign train loss: 0.0011191974626854062

Current batch: 90
Current benign train accuracy: 1.0
Current benign train loss: 0.0017875051125884056

Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0004556944186333567

Current batch: 110
Current benign train accuracy: 1.0
Current benign train loss: 0.0008457706426270306

Current batch: 120
Current benign train accuracy: 1.0
Current benign train loss: 0.0010088447015732527

Current batch: 130
Current benign train accuracy: 1.0
Current benign tr

In [5]:
#RTX3070
deltatime = str(datetime.timedelta(seconds=t2-t1))
print('Total running time is %s '% deltatime)

Total running time is 2:57:31.090026 
